In [28]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
import time
import pandas as pd

In [34]:
path = "C:/Python Dosyalar/Jupyter/MakineOgrn/Projects/Car Price With Selenium/chromedriver.exe"
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get("https://www.arabam.com/ikinci-el/otomobil?city=85&city=86&take=50")


def dataRetriever(index):

    time.sleep(3) # 3 saniye bekleme. sitenin tamamen yüklenmesi için gerekli olabilir. site tamamen yüklenmediğinde selenium tarafında sıkıntı çıkabilir.

    allCars = browser.find_elements(By.TAG_NAME, 'tr') # sayfadaki <tr> tag'ine sahip elementleri bulup allCars değişkenine atar. <tr> tag'ine sahip elementler = sayfadaki tüm arabalar
    ids = []

    # <tr> tag'lerinde id'lere erişip bunları yukarıdaki ids array'ine atmak için gereken döngü. daha sonra bu id'ler ile arabalara tek tek erişeceğiz.
    # buraya her girdiğimizde bu döngü baştan çalışmış olacağından bu döngüyü başka bir fonksiyona yazıp oradan çağırmak istedim ama beceremedim. her şekilde bu kısım baştan çalışmak zorunda kalıyordu.
    for id in allCars:
        ids.append(id.get_attribute("id"))
    while ('' in ids):
        ids.remove('') # array'in içindeki boş string'leri silmek için yazıldı. boş string'lerin olmasının sebebi sayfadaki reklamlar. reklamlarda <tr> tag'ine sahip olduğu için bu listede bulunuyor. onları silmemiz gerek.

    clickedElement = browser.find_element(By.ID, ids[49]) # fonksiyona gönderilen index ile arabalara tek tek eriştiğimiz kısım. aşağıdan sırasıyla 0, 1, ... şeklinde indexler geliyor ve biz arabalara erişiyoruz.
    clickedElement.click() # eriştiğimiz arabaya tıklayıp arabanın sayfasına giriyoruz. 

    specs = browser.find_elements(By.CSS_SELECTOR, '.bcd-list-item') # araba görselinin yanındaki özellikler listesine erişip bu listedekileri specs'e attım. burada bcd-list-item denen şey class adı. buna önce By.CLASS_NAME ile erişmeye çalıştım ama hata aldım. bende class adının başına .(nokta) koyup By.CSS_SELECTOR ile eriştim.
    prices = browser.find_elements(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div/div[1]/div[1]/div[2]/div[1]/div/span') # arabanın fiyatına erişim

    details = []
    price = []

    for i in prices:
        price.append(i.text) # fiyatı yukarıda oluşturalan boş listeye ekleme.
        
    for i in specs:
        details.append(i.text) # özellikleri yukarıda oluşturalan boş listeye ekleme.

    # array'lere eklenen verileri düzenlemek gerektiği için biraz kesme biçme işlemi yapmak gerek :)
    infos = []
    for x in details:
        infos.append(x.replace("\n", "")) # details'in içindeki \n'leri boş string ile değiştirme.
        
    infos = [item.split(':') for item in infos] # \n'lerden arınmış infos isimli listeye erişip :(iki noktalardan) kurtulma.

    i = 0
    rows = []
    column = []

    # şu an infos listesinin her elemanı 2 elemanlı şekilde. mesela ['İlan No', '20818394'] gibi. bunları başlık ve o başlığa karşılık gelen özelliklere ayırmak için kullanılan döngü
    while i <= len(infos)-1:
        rows.append(infos[i][1])
        column.append(infos[i][0])
        i += 1
    column.append("TL") # aldığımız fiyatıda özelliklere ekleyeceğimiz için TL isimli bir sütun oluşturmalıyız.

    strippedPrice = price[0].strip() # aldığımız fiyatın başındaki ve sonundaki boşluklardan kurtulduk
    strippedPrice = strippedPrice.replace(' TL', '') # sadece sayının elimizde kalması için TL'yi kaldırıp yerine boş string getirdik
    rows.append(strippedPrice) # özelliklerimizin içine fiyatı ekledik

    dfContents = pd.DataFrame(rows).T # dataframe'e dönüşüm. sonundaki T, transpoz alması için. bu sayede verileri daha rahat okuyabiliriz.
    dfContents.columns = column # sütün isimlerini yukarıda oluşturduğumuz başlıklar yaptık

    dfContents.to_csv(r"cars.csv",encoding="utf-8-sig",index=False, mode="a") # dataframe'i csv yaptık

    time.sleep(3)
    browser.execute_script("window.history.go(-1)") # tüm işlemler bittiğinde önceki sayfaya dönüş. çünkü diğer arabaya geçeceğiz.

def cars():
    index = 0 # ids array'inin 0. elementine gidip ilk aracın id'sine ulaştık.
    
    # bu döngüde tek tek sayfadaki 50. araca kadar gideceğiz
    while index<=50: 
        try: 
            dataRetriever(index)
            index += 1
        
        # bazı araçlarda eksik bilgi olduğu için IndexError'u except etme.
        # aşağıdaki except'lerde yapılan şey: hata varsa index'i arttır önceki sayfaya dönüp devam et.
        except IndexError:
            index += 1
            browser.execute_script("window.history.go(-1)")
            pass
        # bazı durumlarda nedense araba tıklayamadığı için oluşan hatayı except etme. gerçi son seferde böyle bir problem olmadı. muhtemelen bir daha bu hata oluşmayacak. çünkü arabalara id'ler ile ulaşmadan önce başka şekillerde ulaşmaya çalıştım. bu kısım kaldırılabilir.
        except ElementClickInterceptedException:
            index += 1
            browser.execute_script("window.history.go(-1)")
            pass
        # bazı durumlarda nedense olan arabayı bulamadığı için oluşan hatayı except etme. gerçi son seferde böyle bir problemde olmadı. muhtemelen bir daha bu hata oluşmayacak. çünkü arabalara id'ler ile ulaşmadan önce başka şekillerde ulaşmaya çalıştım. bu kısımda kaldırılabilir.
        except NoSuchElementException:
            index += 1
            browser.execute_script("window.history.go(-1)")
            pass

    # eğer index 51 olmuşsa yani o sayfadaki arabalar bitmişse sonraki sayfaya geçiş
    if index == 51: 
        browser.execute_script("window.history.go(+1)") # dataRetriever fonksiyonun sonunda önceki sayfaya dönmüştük. mesela 2.sayfanın sonuna geldiğimizde dataRetriever fonksiyonu çalıştığında 1.sayfaya dönmüş oluyoruz. ama bizim 3.sayfaya geçmemiz gerek. bunun 2.sayfaya tekrar dönmeliyiz. bu satır bu işe yarıyor. 
        nextPage = browser.find_element(By.XPATH, '//*[@id="pagingNext"]') # sayfanın en altındaki 'sonraki sayfa' butonuna erişim
        nextPage.click() # butona tıklama
        cars() # içinde bulunduğumuz fonksiyonu tekrar çağırıp, geçtiğimiz sonraki sayfanın başından arabalara erişmeye başlama.
        

cars()
    
        

In [19]:
cars = pd.read_csv("cars.csv")
cars

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
1,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
2,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
3,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
4,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
4911,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
4912,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000
4913,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL


In [20]:
import numpy as np
dropList = np.arange(1, len(cars.index)-1, 2)
cars.drop(dropList,axis=0,inplace=True)
cars

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
2,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
4,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
6,20861832,27 Ağustos 2022,Skoda,SuperB,1.6 TDI Active,2017,154.000 km,Yarı Otomatik,Dizel,Hatchback/5,1598 cc,120 hp,Önden Çekiş,"4,1 lt",66 lt,Tamamı orjinal,Takasa Uygun Değil,Sahibinden,650.000
8,20861631,27 Ağustos 2022,Honda,Accord,2.0 Executive,2011,132.500 km,Otomatik,Benzin,Sedan,1998 cc,156 hp,Önden Çekiş,"7,3 lt",65 lt,4 boyalı,Takasa Uygun Değil,Sahibinden,445.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4906,20359095,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,119.639 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,"2 değişen, 1 boyalı",Takasa Uygun,Galeriden,490.000
4908,20359066,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,95.082 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,2 boyalı,Takasa Uygun,Galeriden,499.000
4910,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
4912,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000


In [21]:
cars.reset_index(drop=True, inplace=True)
cars

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
1,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
2,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
3,20861832,27 Ağustos 2022,Skoda,SuperB,1.6 TDI Active,2017,154.000 km,Yarı Otomatik,Dizel,Hatchback/5,1598 cc,120 hp,Önden Çekiş,"4,1 lt",66 lt,Tamamı orjinal,Takasa Uygun Değil,Sahibinden,650.000
4,20861631,27 Ağustos 2022,Honda,Accord,2.0 Executive,2011,132.500 km,Otomatik,Benzin,Sedan,1998 cc,156 hp,Önden Çekiş,"7,3 lt",65 lt,4 boyalı,Takasa Uygun Değil,Sahibinden,445.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,20359095,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,119.639 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,"2 değişen, 1 boyalı",Takasa Uygun,Galeriden,490.000
2454,20359066,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,95.082 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,2 boyalı,Takasa Uygun,Galeriden,499.000
2455,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
2456,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000


In [23]:
cars.reset_index(drop=True, inplace=True)
cars

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
1,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
2,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
3,20861832,27 Ağustos 2022,Skoda,SuperB,1.6 TDI Active,2017,154.000 km,Yarı Otomatik,Dizel,Hatchback/5,1598 cc,120 hp,Önden Çekiş,"4,1 lt",66 lt,Tamamı orjinal,Takasa Uygun Değil,Sahibinden,650.000
4,20861631,27 Ağustos 2022,Honda,Accord,2.0 Executive,2011,132.500 km,Otomatik,Benzin,Sedan,1998 cc,156 hp,Önden Çekiş,"7,3 lt",65 lt,4 boyalı,Takasa Uygun Değil,Sahibinden,445.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,20359095,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,119.639 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,"2 değişen, 1 boyalı",Takasa Uygun,Galeriden,490.000
1998,20359066,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,95.082 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,2 boyalı,Takasa Uygun,Galeriden,499.000
1999,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
2000,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000


In [25]:
cars.drop_duplicates(subset="İlan No", keep=False, inplace=True)
cars

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
1,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
2,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
3,20861832,27 Ağustos 2022,Skoda,SuperB,1.6 TDI Active,2017,154.000 km,Yarı Otomatik,Dizel,Hatchback/5,1598 cc,120 hp,Önden Çekiş,"4,1 lt",66 lt,Tamamı orjinal,Takasa Uygun Değil,Sahibinden,650.000
4,20861631,27 Ağustos 2022,Honda,Accord,2.0 Executive,2011,132.500 km,Otomatik,Benzin,Sedan,1998 cc,156 hp,Önden Çekiş,"7,3 lt",65 lt,4 boyalı,Takasa Uygun Değil,Sahibinden,445.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,20359095,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,119.639 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,"2 değişen, 1 boyalı",Takasa Uygun,Galeriden,490.000
1998,20359066,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,95.082 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,2 boyalı,Takasa Uygun,Galeriden,499.000
1999,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
2000,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000


In [26]:
cars.to_csv('cars-last.csv', encoding='utf-8-sig', index=False)

In [27]:
read = pd.read_csv('cars-last.csv')
read

,İlan No,İlan Tarihi,Marka,Seri,Model,Yıl,Kilometre,Vites Tipi,Yakıt Tipi,Kasa Tipi,Motor Hacmi,Motor Gücü,Çekiş,Ort. Yakıt Tüketimi,Yakıt Deposu,Boya-değişen,Takasa Uygun,Kimden,TL
0,20862633,27 Ağustos 2022,Fiat,Egea,1.4 Fire Street,2021,27.000 km,Düz,Benzin,Hatchback/5,1368 cc,95 hp,Önden Çekiş,"6,6 lt",50 lt,Tamamı orjinal,Takasa Uygun,Galeriden,362.000
1,20862380,27 Ağustos 2022,Renault,Clio,1.0 TCe Touch,2021,21.000 km,Otomatik,Benzin,Hatchback/5,999 cc,100 hp,Önden Çekiş,5 lt,42 lt,Belirtilmemiş,Takasa Uygun,Galeriden,489.500
2,20862220,27 Ağustos 2022,Renault,Fluence,1.5 dCi Icon,2015,121.000 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"4,4 lt",60 lt,Tamamı orjinal,Takasa Uygun,Galeriden,422.000
3,20861832,27 Ağustos 2022,Skoda,SuperB,1.6 TDI Active,2017,154.000 km,Yarı Otomatik,Dizel,Hatchback/5,1598 cc,120 hp,Önden Çekiş,"4,1 lt",66 lt,Tamamı orjinal,Takasa Uygun Değil,Sahibinden,650.000
4,20861631,27 Ağustos 2022,Honda,Accord,2.0 Executive,2011,132.500 km,Otomatik,Benzin,Sedan,1998 cc,156 hp,Önden Çekiş,"7,3 lt",65 lt,4 boyalı,Takasa Uygun Değil,Sahibinden,445.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,20359095,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,119.639 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,"2 değişen, 1 boyalı",Takasa Uygun,Galeriden,490.000
1998,20359066,22 Ağustos 2022,Renault,Megane,1.5 dCi Touch,2017,95.082 km,Yarı Otomatik,Dizel,Sedan,1461 cc,110 hp,Önden Çekiş,"3,8 lt",49 lt,2 boyalı,Takasa Uygun,Galeriden,499.000
1999,19940022,22 Ağustos 2022,Fiat,Linea,1.3 Multijet Active Plus,2011,192.000 km,Düz,Dizel,Sedan,1248 cc,90 hp,Önden Çekiş,"4,9 lt",45 lt,"1 değişen, 1 boyalı",Takasa Uygun Değil,Sahibinden,250.000
2000,20824715,22 Ağustos 2022,Opel,Astra,1.3 CDTI Edition,2013,252.000 km,Düz,Dizel,Sedan,1248 cc,95 hp,Önden Çekiş,"4,2 lt",56 lt,"1 değişen, 11 boyalı",Takasa Uygun,Sahibinden,258.000
